In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import os

task_names = [
    "bernoulli_glm",
    "gaussian_linear",
    "gaussian_linear_uniform",
    "gaussian_mixture",
    "two_moons",
]

values_dfs = {}
std_dfs = {}

for task_name in task_names:
    result_dir = os.path.join("results", task_name)
    values_dfs[task_name] = pd.read_csv(os.path.join(result_dir, "values.csv"), index_col=0)
    std_dfs[task_name] = pd.read_csv(os.path.join(result_dir, "std.csv"), index_col=0)

In [3]:
full_values = pd.concat(values_dfs)
full_stds = pd.concat(std_dfs)

In [4]:
print(full_values.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  column_format="c" * len(full_values.columns),
))

print(full_stds.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  column_format="c" * len(full_stds.columns)
))

\begin{tabular}{cccccc}
\toprule
$\alpha$ & Box & PTC-B & Ellipsoid & PTC-E & CPO \\
\midrule
0.05 & -6.00 & -46.56 & 0.00 & -45.01 & 4.38 \\
0.05 & 0.00 & -0.01 & 0.00 & 0.00 & 0.00 \\
0.05 & 0.00 & -0.01 & 0.00 & -0.00 & -0.03 \\
0.05 & 0.00 & -3.21 & 0.00 & -3.23 & -22.39 \\
0.05 & 0.00 & -0.21 & 0.00 & -0.19 & 0.00 \\
\bottomrule
\end{tabular}

\begin{tabular}{cccccc}
\toprule
$\alpha$ & Box & PTC-B & Ellipsoid & PTC-E & CPO \\
\midrule
0.05 & 0.00 & 22.27 & 0.00 & 19.13 & 0.54 \\
0.05 & 0.00 & 0.02 & 0.00 & 0.00 & 0.00 \\
0.05 & 0.00 & 0.04 & 0.00 & 0.01 & 0.08 \\
0.05 & 0.00 & 3.23 & 0.00 & 3.07 & 30.95 \\
0.05 & 0.00 & 0.27 & 0.00 & 0.28 & 0.00 \\
\bottomrule
\end{tabular}



In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
import numpy as np

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import sbibm

import os
import pickle

In [6]:
task_name = "slcp"

task = sbibm.get_task(task_name)
prior = task.get_prior_dist()
simulator = task.get_simulator()

In [12]:
device = "cuda:0"
cached_fn = os.path.join("trained", f"{task_name}.nf")
with open(cached_fn, "rb") as f:
    encoder = pickle.load(f)
encoder.to(device)

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=10, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer): Linear(in_features=8, out_features=50, bias=True)
              (linear_layers): ModuleList(
                (0-1): 2 x Linear(in_features=50, out_features=50, bias=True)
              )
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): Linear(in_features=50, out_features=87, bias=True)
        )
      )
      (1): LULinear()
      (2): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=11, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer

In [18]:
c = prior.sample((10,))
x = simulator(c).to(device)
c = c.to(device)

In [25]:
encoder.sample(5, x)

tensor([[[-4.5319e-01,  1.5772e-01, -1.2418e+00,  2.1076e+00,  2.3707e+00],
         [-7.1272e-01,  2.8112e+00, -1.3999e+00,  1.4454e+00,  2.3881e+00],
         [-3.9279e-01,  2.5735e+00,  1.2463e+00, -1.6984e+00,  1.2985e+00],
         [-9.2170e-01,  1.4552e+00, -1.6368e+00,  1.8967e+00,  2.0997e+00],
         [-2.4356e+00,  1.2336e+00, -1.4954e+00, -1.6659e+00,  1.7538e+00]],

        [[-5.5357e-01,  2.4265e-02, -1.8028e+00,  2.6282e+00,  1.9291e+00],
         [-2.3763e+00,  7.6003e-03,  1.9184e+00, -3.0059e+00,  1.8025e+00],
         [-3.5926e-01,  2.2549e+00,  2.5962e+00, -2.3844e+00,  1.2497e+00],
         [ 1.0904e+00,  3.0178e+00,  1.7545e+00, -3.0406e+00,  2.1776e+00],
         [-7.8171e-01,  1.9912e+00, -1.7818e+00, -2.6858e+00,  2.8235e+00]],

        [[-2.4810e+00, -9.6669e-01, -1.1110e+00,  2.8203e-01, -1.0277e+00],
         [-1.1957e+00, -7.8689e-01, -2.6782e+00,  1.2707e-01,  2.1655e+00],
         [ 8.0618e-01, -9.1244e-01, -2.6276e+00, -2.5432e-01,  1.3139e+00],
        

In [24]:
import os

In [32]:
method_names = ["Box", "PTC-B", "Ellipsoid", "PTC-E", "CPO"]
task_names = os.listdir("results")

task_name_to_value = {}
for task_name in task_names:
    values = []
    for method_name in method_names:
        fn = os.path.join("results", task_name, f"{method_name}.csv")
        df = pd.read_csv(fn, index_col=0)
        values.append(np.mean(df.values))
    task_name_to_value[task_name] = values

In [40]:
task_name_to_value_df = pd.DataFrame(task_name_to_value, index=method_names).T
print(task_name_to_value_df.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  column_format="c" * len(task_name_to_value),
))

\begin{tabular}{ccccccccc}
\toprule
 & Box & PTC-B & Ellipsoid & PTC-E & CPO \\
\midrule
sir & -0.16 & -0.43 & -0.08 & -0.37 & -0.35 \\
two_moons & 0.00 & 0.00 & 0.00 & 0.00 & -0.03 \\
slcp & 0.00 & 0.00 & 0.00 & 0.00 & 0.01 \\
gaussian_linear_uniform & 0.00 & -0.05 & 0.00 & -0.10 & -0.51 \\
slcp_distractors & 0.00 & 0.00 & 0.00 & 0.00 & 0.03 \\
lotka_volterra & -0.52 & -0.49 & -0.01 & -0.40 & -0.29 \\
bernoulli_glm & 0.00 & -0.05 & 0.00 & -0.04 & -1.16 \\
gaussian_mixture & 0.00 & -3.45 & 0.00 & -3.04 & -66.93 \\
gaussian_linear & 0.00 & 0.00 & 0.00 & 0.00 & 0.03 \\
\bottomrule
\end{tabular}

